In [34]:
#import libraries
import glob, copy, time, os, sys
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import seaborn as sns
import numpy as np
import cv2
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
BATCH_SIZE=64
num_epochs=5
lr=1e-4
class_size=10

In [36]:
transform_train = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.RandomHorizontalFlip(p=0.7), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
transform_valid = transforms.Compose([
    transforms.Resize(size=256),
    transforms.CenterCrop(size =224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
transform_test = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(), 
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [38]:
# #prep the train, validation and test dataset
# torch.manual_seed(2021)
# train = torchvision.datasets.CIFAR10("data/", train=True, download=True, transform=tranform_train) 
# val_size = 10000 
# train_size = len(train) - val_size
# train, val = random_split(train, [train_size, val_size]) 
# test = torchvision.datasets.CIFAR10("data/", train=False, download=True, transform=tranform_test) 

# #  train, val and test datasets to the dataloader
# train_loader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
# val_loader = DataLoader(val, batch_size=BATCH_SIZE, shuffle=False)

In [39]:
# We only have train and test data folder. So, we separate some of the images from the train
# set and assign that as validation dataset.
parent_path = '/home/sysadm/Desktop/3ml/chest_xray'
train_paths = glob.glob(os.path.join(parent_path, 'train/*/*'))
train_class = [s.split('/')[-2] for s in train_paths]
test_paths = glob.glob(os.path.join(parent_path, 'test/*/*'))
test_class = [s.split('/')[-2] for s in test_paths]

In [40]:
train_paths, valid_paths, train_class, valid_class = train_test_split(train_paths, train_class, test_size=0.11, random_state=12345)
print("\nTrain size: {}\nValid size: {}\nTest size: {}".format(len(train_paths), len(valid_paths), len(test_paths)))


Train size: 4656
Valid size: 576
Test size: 624


Define Dataset Class

In [41]:
class ChestXRay(Dataset):
    def __init__(self, img_paths, transform=False):
        self.img_paths = img_paths
        self.transform = transform
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = cv2.imread(img_path)
        label = img_path.split('/')[-2]
        label = class_to_idx[label]
        return img, label

Create Dataset

In [26]:
X_train = ChestXRay(train_paths, transform = transform_train)
X_valid = ChestXRay(valid_paths, transform = transform_valid)
X_test = ChestXRay(test_paths, transform = transform_test)

In [29]:
print('The shape of tensor for 50th image in train dataset: ',X_train[48][0].shape)
print('The label for 50th image in train dataset: ',X_train[48][1])

The shape of tensor for 50th image in train dataset:  (1684, 1742, 3)
The label for 50th image in train dataset:  4655


In [32]:
train_loader = DataLoader(X_train, batch_size = BATCH_SIZE, shuffle = True)
valid_loader = DataLoader(X_valid, batch_size = BATCH_SIZE, shuffle = True)
test_loader = DataLoader(X_test, batch_size = BATCH_SIZE, shuffle = True)
loaders = {'train': train_dataloader, 'valid': valid_dataloader, 'test': test_dataloader}

In [33]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
for images, _ in train_loader:
    print('images.shape:', images.shape)
    plt.figure(figsize=(16,8))
    plt.axis('off')
    plt.imshow(make_grid(images, nrow=16).permute((1, 2, 0)))
    break

RuntimeError: stack expects each tensor to be equal size, but got [1728, 2088, 3] at entry 0 and [1040, 1380, 3] at entry 1